##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Título

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/bigtable"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este notebook apresenta o uso básico e os recursos do módulo `tensorflow_io.bigtable`. Você deve conhecer estes temas antes de prosseguir:

1. [Como criar um projeto do GCP.](https://console.cloud.google.com/cloud-resource-manager)
2. [Como instalar o Cloud SDK para Bigtable](https://cloud.google.com/bigtable/docs/installing-cloud-sdk)
3. [Visão geral da cbt tool](https://cloud.google.com/bigtable/docs/cbt-overview)
4. [Como usar o emulador](https://cloud.google.com/bigtable/docs/emulator)

**Observação**: o Jupyter executa linhas com o prefixo `!` como comandos do shell e interpola variáveis do Python com o prefixo `$` com esses comandos.

## Configuração

In [ ]:
!pip install tensorflow-io

**Observação**: ao executar a célula abaixo, será solicitado que você faça login no Google Cloud.

In [ ]:
!mkdir /tools/google-cloud-sdk/.install
!gcloud --quiet components install beta cbt bigtable
!gcloud init

Neste exemplo, o emulador Bigtable é usado. Se você já tiver configurado sua instância do Bigtable e preenchido com valores, pule estas etapas e vá diretamente para a seção Início rápido.

Inicie o emulador em segundo plano.

In [ ]:
import os
import subprocess
_emulator = subprocess.Popen(['/tools/google-cloud-sdk/bin/gcloud', 'beta', 'emulators', 'bigtable', 'start', '--host-port=127.0.0.1:8086'],
                                      stdout=subprocess.DEVNULL,
                                      stderr=subprocess.DEVNULL, bufsize=0)

Crie uma tabela

In [ ]:
%env BIGTABLE_EMULATOR_HOST=127.0.0.1:8086
!cbt -project "test-project" -instance "test-instance" createtable t1 families=cf1 splits=row-a,row-h,row-p,row-z
!cbt -project "test-project" -instance "test-instance" ls


Preencha a tabela com valores

In [ ]:
!cbt -project "test-project" -instance "test-instance" set t1 row-a cf1:c1=A
!cbt -project "test-project" -instance "test-instance" set t1 row-b cf1:c1=B
!cbt -project "test-project" -instance "test-instance" set t1 row-c cf1:c1=C
!cbt -project "test-project" -instance "test-instance" set t1 row-d cf1:c1=D
!cbt -project "test-project" -instance "test-instance" set t1 row-e cf1:c1=E
!cbt -project "test-project" -instance "test-instance" set t1 row-f cf1:c1=F
!cbt -project "test-project" -instance "test-instance" set t1 row-g cf1:c1=G
!cbt -project "test-project" -instance "test-instance" set t1 row-h cf1:c1=H
!cbt -project "test-project" -instance "test-instance" set t1 row-i cf1:c1=I
!cbt -project "test-project" -instance "test-instance" set t1 row-j cf1:c1=J
!cbt -project "test-project" -instance "test-instance" set t1 row-k cf1:c1=K
!cbt -project "test-project" -instance "test-instance" set t1 row-l cf1:c1=L
!cbt -project "test-project" -instance "test-instance" set t1 row-m cf1:c1=M
!cbt -project "test-project" -instance "test-instance" set t1 row-n cf1:c1=N
!cbt -project "test-project" -instance "test-instance" set t1 row-o cf1:c1=O
!cbt -project "test-project" -instance "test-instance" set t1 row-p cf1:c1=P
!cbt -project "test-project" -instance "test-instance" set t1 row-q cf1:c1=Q
!cbt -project "test-project" -instance "test-instance" set t1 row-r cf1:c1=R
!cbt -project "test-project" -instance "test-instance" set t1 row-s cf1:c1=S
!cbt -project "test-project" -instance "test-instance" set t1 row-t cf1:c1=T
!cbt -project "test-project" -instance "test-instance" set t1 row-u cf1:c1=U
!cbt -project "test-project" -instance "test-instance" set t1 row-v cf1:c1=V
!cbt -project "test-project" -instance "test-instance" set t1 row-w cf1:c1=W
!cbt -project "test-project" -instance "test-instance" set t1 row-x cf1:c1=X
!cbt -project "test-project" -instance "test-instance" set t1 row-y cf1:c1=Y
!cbt -project "test-project" -instance "test-instance" set t1 row-z cf1:c1=Z

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_io as tfio
import random

random.seed(10)

## Início rápido

Primeiro você precisa criar um cliente e uma tabela da qual deseja ler.

In [ ]:
# If using your bigtable instance replace the project_id, instance_id 
# and the name of the table with suitable values.

client = tfio.bigtable.BigtableClient(project_id="test-project", instance_id="test-instance")
train_table = client.get_table("t1")

Excelente! Agora, você pode criar um dataset do TensorFlow que lerá os dados da tabela.

Para fazer isso, é preciso fornecer o tipo de dados que você deseja ler, listar os nomes das colunas no formato `column_family:column_name` e um row_set (conjunto de linhas) que você deseja ler.

Para criar um row_set, use os métodos utilitários fornecidos nos módulos `tensorflow.bigtable.row_set` e `tensorflow.bigtable.row_range`. Um row_set contendo todas as linhas é criado.

Lembre-se de que o Bigtable lê valores em ordem lexicográfica, não na ordem em que eles são colocados. As linhas receberam chaves de linha aleatórias para poderem ser misturadas.

In [ ]:
row_set = tfio.bigtable.row_set.from_rows_or_ranges(tfio.bigtable.row_range.infinite())

train_dataset = train_table.read_rows(["cf1:c1"],row_set, output_type=tf.string)

for tensor in train_dataset:
  print(tensor)

Pronto! Parabéns!

## Leitura paralela

O dataset tem suporte à leitura paralela do Bigtable. Para fazer isso, use o método `parallel_read_rows` e especifique `num_parallel_calls` (número de chamadas paralelas) como um argumento. Quando esse método é chamado, o trabalho é dividido entre os workers com base em SampleRowKeys.

**Observação**: lembre-se de que, na leitura em paralelo, as linhas não serão lidas em nenhuma ordem específica.

In [ ]:
for tensor in train_table.parallel_read_rows(["cf1:c1"],row_set=row_set, num_parallel_calls=2):
  print(tensor)

## Lendo row_keys específicas

Para ler dados do Bigtable, você pode especificar um conjunto ou um intervalo de linhas, ou uma combinação dos dois.

O método `read_rows` espera que você forneça um RowSet. Você pode criar um RowSet a partir de chaves de linha específicas ou de RowRanges (intervalos de linha) da seguinte forma:

In [ ]:
row_range_below_300 = tfio.bigtable.row_range.right_open("row000", "row300")

my_row_set = tfio.bigtable.row_set.from_rows_or_ranges(row_range_below_300, "row585", "row832")
print(my_row_set)

Esse row_set conteria um intervalo de linhas `[row000, row300]` e também as linhas row585 e row832.

Você também pode criar um row_set com um intervalo infinito, intervalo vazio ou um prefixo. Além disso, é possível fazer a interseção dele com um row_range.

In [ ]:
my_truncated_row_set = tfio.bigtable.row_set.intersect(my_row_set,
                                         tfio.bigtable.row_range.right_open("row200", "row700"))
print(my_truncated_row_set)

## Especificando uma versão de um valor

O Bigtable permite manter diversos valores em uma célula com timestamps diferentes. Você pode especificar qual versão deseja escolher usando os filtros de versão. Porém, você pode recuperar somente um vetor bidimensional usando o conector tensorflow.bigtable, então o filtro `latest` (mais recente) sempre é anexado ao filtro de versão especificado pelo usuário. Isso significa que, se mais de um valor para uma célula passar pelo filtro fornecido, o mais recente será usado.

Você pode usar o filtro `latest` passando o valor mais recente ou pode especificar um intervalo de tempo, que pode ser fornecido como objetos DateTime do Python ou como um número representando os segundos ou microssegundos desde a época.

In [ ]:
from datetime import datetime

start = datetime(2020, 10, 10, 12, 0, 0)
end = datetime(2100, 10, 10, 13, 0, 0)
from_datetime = tfio.bigtable.filters.timestamp_range(start, end)
from_posix_timestamp = tfio.bigtable.filters.timestamp_range(int(start.timestamp()), int(end.timestamp()))
print("from_datetime:", from_datetime)

print("from_posix_timestamp:", from_posix_timestamp)